# Economic Model Validation

## Before you begin

::: {.callout-tip}
### New to ValidMind? 
For access to all features available in this notebook, create a free ValidMind account. 

Signing up is FREE — [**Sign up now!**](https://app.prod.validmind.ai)
:::

If you encounter errors due to missing modules in your Python environment, install the modules with `pip install`, and then re-run the notebook. For more help, refer to [Installing Python Modules](https://docs.python.org/3/installing/index.html).

## Install the client library

In [ ]:
# %pip install -q validmind

## Initialize the client library

ValidMind generates a unique _code snippet_ for each registered model to connect with your developer environment. You initialize the client library with this code snippet, which ensures that your documentation and tests are uploaded to the correct model when you run the notebook.

Get your code snippet:

1. In a browser, log into the [Platform UI](https://app.prod.validmind.ai).

2. In the left sidebar, navigate to **Model Inventory** and click **+ Register new model**.

3. Enter the model details and click **Continue**. ([Need more help?](https://docs.validmind.ai/guide/register-models-in-model-inventory.html))

4. Go to **Getting Started** and click **Copy snippet to clipboard**.

Next, replace this placeholder with your own code snippet:

In [ ]:
import validmind as vm

vm.init(
    api_host="https://api.prod.validmind.ai/api/v1/tracking",
    api_key="...",
    api_secret="...",
    project="..."
)

## Introduction

## Setup

### Import Libraries

In [ ]:
# Load API key and secret from environment variables
%load_ext dotenv
%dotenv .env

from IPython.display import HTML
from notebooks.probability_of_default.helpers.Developer import Developer
from notebooks.probability_of_default.helpers.economic_model import *

### Input Parameters

In [ ]:
target_column = "DRSFRMACBS"

### Load Datasets and Models

In [ ]:
developer = Developer()
macro_model = developer.load_objects_from_pickle(
    "datasets/macroeconomic_data_and_models.pkl")

df_raw = macro_model["df_raw"]
df_preparation = macro_model["df_prepared"]

df_train = macro_model["df_train"]
df_test = macro_model["df_test"]
model_fit = macro_model["model_fit"]

df_train_final = macro_model["df_train_final"]
df_test_final = macro_model["df_test_final"]
model_fit_final = macro_model["model_fit_final"]

### Create ValidMind Datasets and Models

In [ ]:
from validmind.vm_models.test_context import TestContext

#  Create ValidMind Datasets
vm_df_raw = vm.init_dataset(
    dataset=df_raw,
    target_column=target_column)
vm_df_preparation = vm.init_dataset(
    dataset=df_preparation,
    target_column=target_column)
vm_df_train = vm.init_dataset(
    dataset=df_train,
    target_column=target_column)
vm_df_test = vm.init_dataset(
    dataset=df_test,
    target_column=target_column)
vm_df_train_final = vm.init_dataset(
    dataset=df_train_final,
    target_column=target_column)
vm_df_test_final = vm.init_dataset(
    dataset=df_test_final,
    target_column=target_column)

#  Create ValidMind Models
vm_model_fit = vm.init_model(
    model=model_fit,
    train_ds=vm_df_train,
    test_ds=vm_df_test)
vm_model_fit_final = vm.init_model(
    model=model_fit_final,
    train_ds=vm_df_train_final,
    test_ds=vm_df_test_final)

#  Create test contexts
test_context_raw = TestContext(dataset=vm_df_raw)
test_context_preparation = TestContext(dataset=vm_df_preparation)
test_context_train = TestContext(dataset=vm_df_train)
test_context_test = TestContext(dataset=vm_df_test)
test_context_train_final = TestContext(dataset=vm_df_train_final)
test_context_test_final = TestContext(dataset=vm_df_test_final)

test_context_model = TestContext(model=vm_model_fit)
test_context_models = TestContext(models=[vm_model_fit, vm_model_fit_final])

## Model Validation

### Raw Data

In [ ]:
from validmind.tests.data_validation.TimeSeriesMissingValues import TimeSeriesMissingValues

params = {"min_threshold": 2}

metric = TimeSeriesMissingValues(test_context_raw, params)
metric.run()
metric.result.log()
metric.result.show()

In [ ]:
from validmind.tests.data_validation.TimeSeriesOutliers import TimeSeriesOutliers

params = {"zscore_threshold": 3}

metric = TimeSeriesOutliers(test_context_raw, params)
metric.run()
metric.result.log()
metric.result.show()

In [ ]:
from validmind.tests.data_validation.TimeSeriesFrequency import TimeSeriesFrequency

metric = TimeSeriesFrequency(test_context_raw)
metric.run()
metric.result.log()
metric.result.show()

### Prepared Data 

In [ ]:
from validmind.tests.data_validation.TimeSeriesMissingValues import TimeSeriesMissingValues

params = {"min_threshold": 2}

metric = TimeSeriesMissingValues(test_context_preparation, params)
metric.run()
metric.result.log()
metric.result.show()

In [ ]:
from validmind.tests.data_validation.TimeSeriesFrequency import TimeSeriesFrequency

metric = TimeSeriesFrequency(test_context_preparation)
metric.run()
metric.result.log()
metric.result.show()

### Train Data 

In [ ]:
from validmind.tests.data_validation.TimeSeriesLinePlot import TimeSeriesLinePlot

metric = TimeSeriesLinePlot(test_context_train)
metric.run()
metric.result.log()
metric.result.show()

In [ ]:
metric = TimeSeriesLinePlot(test_context_test)
metric.run()
metric.result.log()
metric.result.show()

In [ ]:
from validmind.tests.data_validation.LaggedCorrelationHeatmap import LaggedCorrelationHeatmap

metric = LaggedCorrelationHeatmap(test_context_train)
metric.run()
metric.result.log()
metric.result.show()

In [ ]:
from validmind.tests.data_validation.EngleGrangerCoint import EngleGrangerCoint

metric = EngleGrangerCoint(test_context_train)
metric.run()
metric.result.log()
metric.result.show()

### Model Training

In [ ]:
from validmind.tests.data_validation.DatasetSplit import DatasetSplit

metric = DatasetSplit(test_context_model)
metric.run()
metric.result.log()
metric.result.show()

In [ ]:
from validmind.tests.model_validation.ModelMetadata import ModelMetadata

metric = ModelMetadata(test_context_model)
metric.run()
metric.result.log()
metric.result.show()

In [ ]:
from validmind.tests.model_validation.statsmodels.RegressionCoeffsPlot import RegressionCoeffsPlot

metric = RegressionCoeffsPlot(test_context_models)
metric.run()
metric.result.log()
metric.result.show()

In [ ]:
from validmind.tests.model_validation.statsmodels.RegressionModelsCoeffs import RegressionModelsCoeffs

metric = RegressionModelsCoeffs(test_context_models)
metric.run()
metric.result.log()
metric.result.show()

### Model Evaluation

In [ ]:
from validmind.tests.model_validation.statsmodels.RegressionModelsPerformance import RegressionModelsPerformance

metric = RegressionModelsPerformance(test_context_models)
metric.run()
metric.result.log()
metric.result.show()